# Практикуемся в ВКР

## 1. Импорт библиотек 

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from PIL import Image
from scipy.signal import convolve2d
from sklearn.utils import shuffle

import torch, torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F

import time

/Users/nnn/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/nnn/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <F263C8ED-23C1-35DD-BD33-2CD667C0ED1D> /Users/nnn/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <F8622D92-25A9-3A61-A089-C917FDA36C1B> /Users/nnn/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## 2. Загрузка данных 

In [2]:
clear_data_path = 'data/clear_data'
stego_data_path = 'data/stego_data'

pictures_names = os.listdir('data/clear_data')

for name in tqdm(pictures_names):
    if not name.startswith('.'):
        payload = 0.4               
        params = -1              
        
        filename = os.path.join(clear_data_path, name)
        cover = Image.open(filename)
        cover = cover.resize((256,256))
        
        cover = cover.convert('L')
        cover.save(filename)

100%|██████████████████████████████████████| 5038/5038 [00:06<00:00, 819.85it/s]


## 4. Структурирование данных 

In [3]:
clear_paths = list(map(lambda x: os.path.join(clear_data_path, x), os.listdir(clear_data_path)))
pictures_clear_df = pd.DataFrame({
    "picture_link": clear_paths,
    "is_changed": np.zeros(len(clear_paths), dtype=int)
})

stego_paths = list(map(lambda x: os.path.join(stego_data_path, x), os.listdir(stego_data_path)))
pictures_graphed_df = pd.DataFrame({
    "picture_link": stego_paths,
    "is_changed": np.ones(len(stego_paths), dtype=int)
})

data = shuffle(pd.concat([pictures_clear_df, pictures_graphed_df]))
data.reset_index(drop=True, inplace=True)

In [4]:
data = data[data['picture_link'] != "data/clear_data/.DS_Store"]
data = data[data['picture_link'] != "data/stego_data/.DS_Store"]
data.to_csv("data.csv", index=False)

In [5]:
class Dataset(Dataset):
    """Описантельный класс датасета для удобной работы с ним"""

    def __init__(self, csv_file=None, transform=None):
        """
            Args:
                csv_file (string): Путь к csv файлу с разметкой
                transform (callable, optional): Опционально, трансформации применяемые к картинкам
        """
        
        self.annotations = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return self.annotations.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_info = self.annotations.iloc[idx]
        image = io.imread(img_info[0])
        label = img_info[1]
        
        if self.transform:
            image = self.transform(image)
                
        sample = torch.tensor([image], dtype=torch.float32), label

        return sample

In [6]:
transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((256, 256)), transforms.ToTensor()])
train_dataset = Dataset(csv_file='data.csv')

train_part = np.arange(0, int(len(train_dataset) * 0.7))
val_part = np.arange(int(len(train_dataset) * 0.7), int(len(train_dataset) * 0.9))
test_part = np.arange(int(len(train_dataset) * 0.9), len(train_dataset))

sampler_to_train = torch.utils.data.SubsetRandomSampler(train_part)
sampler_to_val = torch.utils.data.SubsetRandomSampler(val_part)
sampler_to_test = torch.utils.data.SubsetRandomSampler(test_part)

train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler_to_train) 
val_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler_to_val)
test_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler_to_test)